In [1]:
# import qtcore,qwidgwts,qtgui then ui_pyinsta(this is the ui file ) then set up class to run the ui file
from PySide6 import QtCore, QtWidgets, QtGui

In [2]:
import sys
import ui_pyinsta


In [3]:
import pymongo
from pymongo import MongoClient

In [4]:
from using_functions import final_follow,final_unfollow,final_postpicture,all_actions

In [5]:
from selenium import webdriver

In [6]:
class MainWindow(QtWidgets.QMainWindow, ui_pyinsta.Ui_MainWindow):
    def __init__(self):
        super().__init__()
        self.ui=ui_pyinsta.Ui_MainWindow()
        self.client=MongoClient()
        self.db=self.client['pyinsta']
        self.ui.setupUi(self)
        self.driver=webdriver.Chrome()
        
        self.ui.Addaccount.setCurrentWidget(self.ui.page_2)
        self.home_page()
        self.accounts_page()
        
        
        
        # button connecting functions
    def home_page(self):
        self.ui.accountspage.clicked.connect(lambda:self.ui.Addaccount.setCurrentWidget(self.ui.page))
        self.ui.accountspage1.clicked.connect(lambda:self.ui.Addaccount.setCurrentWidget(self.ui.page))
        
    def accounts_page(self):
        
        self.ui.pushButton_2.clicked.connect(lambda:self.save_growaccount())
        self.growingaccounts_table()
        
        
    def save_growaccount(self):
        username=self.ui.lineEdit.text()
        password=self.ui.lineEdit_2.text()
        growaccount=self.db['growaccounts']
        growaccount.insert_one({'main_username':username,'password':password})
        self.accounts_page()
        
        
    def growingaccounts_table(self):
        accounts=list(self.db.growaccounts.find())
        self.ui.tableWidget.setColumnCount(4)
        self.ui.tableWidget.setRowCount(len(accounts))
        self.ui.tableWidget.setHorizontalHeaderLabels(["Account", "Settings", "Control Center","Delete"])
        self.ui.tableWidget.horizontalHeader().setSectionResizeMode(QtWidgets.QHeaderView.Stretch)
        self.ui.tableWidget.verticalHeader().setSectionResizeMode( QtWidgets.QHeaderView.Stretch)
        count=0
        for account in accounts:
            self.ui.tableWidget.setItem(count,0, QtWidgets.QTableWidgetItem(str(account["main_username"])))
            setting_button=QtWidgets.QPushButton("Settings")
            setting_button.clicked.connect(lambda:self.settings_page(account['main_username']))
            
            self.ui.tableWidget.setCellWidget(count,1,setting_button)
            control_button=QtWidgets.QPushButton("Control")
            control_button.clicked.connect(lambda:self.control_page(account['main_username']))
            self.ui.tableWidget.setCellWidget(count,2,control_button)
            
            delete_button=QtWidgets.QPushButton("Delete")
            delete_button.clicked.connect(lambda:self.delete_growaccount(account["main_username"]))
            self.ui.tableWidget.setCellWidget(count,3,delete_button)
            
            
     
            count+=1
            
    def settings_page(self,account):
        self.ui.Addaccount.setCurrentWidget(self.ui.page_4)
        imagefolder_link=list(self.db.imagefolder.find({"main_username":account}))
        if imagefolder_link==[]:
            self.ui.lineEdit_6.setPlaceholderText("No folder set")
            self.ui.pushButton_6.clicked.connect(lambda:self.folder_button(account))
        else:
            self.ui.lineEdit_6.setPlaceholderText(imagefolder_link[0]["foldername"])
            self.ui.pushButton_6.clicked.connect(lambda:self.update_folderbutton(account))
        
            self.ui.pushButton_6.setText("Update")
            
        followlimit=list(self.db.followlimit.find({"main_username":account}))
        if followlimit==[]:
            self.ui.lineEdit_3.setPlaceholderText("No limit set")
            self.ui.folowlimit.clicked.connect(lambda:self.followlimit(account))
        else:
            self.ui.lineEdit_3.setPlaceholderText(followlimit[0]["limit"])
            self.ui.folowlimit.clicked.connect(lambda:self.update_followbtn(account))
            self.ui.folowlimit.setText("Update")
        unfollowlimit=list(self.db.unfollowlimit.find({"main_username":account}))
        if unfollowlimit==[]:
            self.ui.lineEdit_5.setPlaceholderText("No limit set")
            self.ui.Change.clicked.connect(lambda:self.unfollowlimit(account))
        else:
            self.ui.lineEdit_5.setPlaceholderText(unfollowlimit[0]["limit"])
            self.ui.Change.clicked.connect(lambda:self.update_unfollowbtn(account))
            self.ui.Change.setText("Update")
            
        self.ui.pushButton_7.clicked.connect(lambda:self.save_captionbutton(account))
        
        
            
        
            
            
        follow_accounts=list(self.db.followaccounts.find({"main_username":account}))
        self.ui.pushButton_4.clicked.connect(lambda:self.add_followaccount(account))
        self.followaccounts_table(follow_accounts)
        self.captions_table(account)
    def followaccounts_table(self,followaccounts):
        print(followaccounts)
        self.ui.tableWidget_2.setColumnCount(2)
        self.ui.tableWidget_2.setRowCount(len(followaccounts))
        self.ui.tableWidget_2.setHorizontalHeaderLabels(["Account","Delete"])
        self.ui.tableWidget_2.horizontalHeader().setSectionResizeMode(QtWidgets.QHeaderView.Stretch)
        self.ui.tableWidget_2.verticalHeader().setSectionResizeMode( QtWidgets.QHeaderView.Stretch)
        count=0
        for account in followaccounts:
            print(account)
            self.ui.tableWidget_2.setItem(count,0, QtWidgets.QTableWidgetItem(str(account["username"])))
           
            
            delete_button=QtWidgets.QPushButton("Delete")
            delete_button.clicked.connect(lambda:self.delete_followaccount(account["username"]))
            self.ui.tableWidget_2.setCellWidget(count,1,delete_button)
            
            
     
            count+=1
        
    def delete_followaccount(self,username):
        self.db.followaccounts.delete_one({"username":username})
        
    def control_page(self,main_username):
        self.ui.Addaccount.setCurrentWidget(self.ui.page_3)
        self.ui.activate_unfollow_2.clicked.connect(lambda:self.all_actions(main_username))
        self.ui.activate_follow.clicked.connect(lambda:self.start_follow(main_username))
        self.ui.activate_unfollow.clicked.connect(lambda:self.start_unfollow(main_username))
        self.ui.activate_post.clicked.connect(lambda:self.start_post(main_username))
        self.ui.poststatus_3.setText("READY")
        self.ui.poststatus_4.setText("READY")
        self.ui.poststatus.setText("READY")
        self.ui.poststatus_2.setText("Coming soon")
        self.initialize_followcount(main_username)
        
        
        
        
        
        
    def all_actions(self,main_username):
        all_actions(self.driver,main_username)
        
    def start_follow(self,main_username):
        final_follow(self.driver,main_username)
        
    def start_unfollow(self,main_username):
        final_unfollow(self.driver,main_username)
        
    def start_post(self,main_username):
        final_postpicture(self.driver,main_username)
    def detailcheck(self,main_username):
        followlimit=self.db['followlimit']
        unfollowlimit=self.db['unfollowlimit']
        imagefolder=self.db['imagefolder']
        followcount=self.db['followcount']
        imagetracker=self.db['imagetracker']
        captiondetails=self.db['captions']
        followlimit=followlimit.find_one({'main_username':main_username})
        unfollowlimit=unfollowlimit.find_one({'main_username':main_username})
        imagefolder=imagefolder.find_one({'main_username':main_username})
        followcount=followcount.find_one({'main_username':main_username})
        imagetracker=imagetracker.find_one({'main_username':main_username})
        captiondetails=captiondetails.find_one({'main_username':main_username})
        if followlimit is None or unfollowlimit is None or imagefolder is None or followcount is None or imagetracker is None or captiondetails is None:
            return False
        else:
            return True
        
    
    
    
    def save_captionbutton(self,main_username):
        caption=self.ui.lineEdit_7.text()
        if caption=="":
            self.ui.lineEdit_7.setPlaceholderText("Please enter a caption")
        else:
            self.save_caption(main_username,caption)
            self.captions_table(main_username)   
                
    def delete_growaccount(self,username):
        self.db.growaccounts.delete_one({"main_username":username})
        self.accounts_page()
        
    def add_followaccount(self,main_username):
        username=self.ui.lineEdit_4.text()
        self.save_followaccount(username,main_username)
        self.settings_page()
        
    def captions_table(self,main_username):
        captions=list(self.db.captions.find({"main_username":main_username}))
        self.ui.tableWidget_3.setColumnCount(2)
        self.ui.tableWidget_3.setRowCount(len(captions))
        self.ui.tableWidget_3.setHorizontalHeaderLabels(["caption","Delete"])
        self.ui.tableWidget_3.horizontalHeader().setSectionResizeMode(QtWidgets.QHeaderView.Stretch)
        self.ui.tableWidget_3.verticalHeader().setSectionResizeMode( QtWidgets.QHeaderView.Stretch)
        count=0
        for caption in captions:
            self.ui.tableWidget_3.setItem(count,0, QtWidgets.QTableWidgetItem(str(caption["caption"])))
           
            
            delete_button=QtWidgets.QPushButton("Delete")
            delete_button.clicked.connect(lambda:self.delete_caption(main_username,caption["caption"]))
            self.ui.tableWidget_3.setCellWidget(count,1,delete_button)
            
     
            count+=1
            
    def save_caption(self,main_username,caption):
        captiondetails=self.db['captions']
        captiondetails.insert_one({'main_username':main_username,'caption':caption})
        self.settings_page(main_username)
    
    def delete_caption(self,main_username,caption):
        captiondetails=self.db['captions']
        captiondetails.delete_one({'main_username':main_username,'caption':caption})
        self.settings_page(main_username)
        
        
    def save_followaccount(self,username,main_username):
        followdetails=self.db['followaccounts']
        followdetails.insert_one({'username':username,'main_username':main_username})
        
        
    def folder_button(self,main_username):
        folderlink=self.ui.lineEdit_6.text()
        self.save_imagefolder(main_username,folderlink)
        self.settings_page(main_username)
        
    def update_folderbutton(self,main_username):
        link=self.ui.lineEdit_6.text()
        self.ui.pushButton_6.clicked.connect(lambda:self.update_imagefolder(main_username,link))
        self.settings_page(main_username)
        
    def update_imagefolder(self,main_username,foldername):
        imagefolder=self.db['imagefolder']
        imagefolder.update_one({'main_username':main_username},{'$set':{'foldername':foldername}})
        self.settings_page(main_username)
        
        
    def save_imagefolder(self,main_username,foldername):
        imagefolder=self.db['imagefolder']
        imagefolder.insert_one({'main_username':main_username,'foldername':foldername})
        self.settings_page(main_username)
        
    def followlimit(self,main_username):
        limit=self.ui.lineEdit_3.text()
        followlimit=self.db['followlimit']
        followlimit.insert_one({'main_username':main_username,'limit':limit})
        self.settings_page(main_username)
        
        

    def unfollowlimit(self,main_username):
        limit=self.ui.lineEdit_5.text()
        unfollowlimit=self.db['unfollowlimit']
        unfollowlimit.insert_one({'main_username':main_username,'limit':limit})
        self.settings_page(main_username)
        
    def follow_btn(self,username):
        self.ui.folowlimit.clicked.connect(lambda:self.followlimit(username))
        self.settings_page(username)
        
    def unfollow_btn(self,username):
        self.ui.Change.connect(self.unfollowlimit(username))
        self.settings_page(username)
        
    def update_followbtn(self,username):
        self.ui.folowlimit.clicked.connect(lambda:self.update_followlimit(username))
        self.settings_page(username)
    
    def update_unfollowbtn(self,username):
        self.ui.Change.clicked.connect(lambda:self.update_unfollowlimit(username))
        self.settings_page(username)
        
    def update_unfollowlimit(self,main_username):
        limit=self.ui.lineEdit_5.text()
        unfollowlimit=self.db['unfollowlimit']
        unfollowlimit.update_one({'main_username':main_username},{'$set':{'limit':limit}})
        self.settings_page(main_username)
    
    def update_followlimit(self,main_username):
        limit=self.ui.lineEdit_3.text()
        followlimit=self.db['followlimit']
        followlimit.update_one({'main_username':main_username},{'$set':{'limit':limit}})
        self.settings_page(main_username)
    def initialize_followcount(self,main_username):
        #fetch existing recordif none create
        followc=self.db['followcounter'].find_one({'main_username':main_username})
        if followc is None:
            followcount=self.db['followcounter']
            followcount.insert_one({'main_username':main_username,'count':0})
            
        else:
            pass
        
        
        
        
    
    
        

        

In [7]:
app = QtWidgets.QApplication([])
window = MainWindow()           
window.show()
sys.exit(app.exec_())  # this is the main loop to run the app

C:\Users\DELL\AppData\Local\Temp\ipykernel_18456\1548868628.py:4: DeprecationWarning: 'exec_' will be removed in the future. Use 'exec' instead.
  sys.exit(app.exec_())  # this is the main loop to run the app


ElementClickInterceptedException: Message: element click intercepted: Element <div class="_ap3a _aaco _aacw _aad6 _aade" dir="auto">...</div> is not clickable at point (624, 386). Other element would receive the click: <div class="x19991ni x1hc1fzr xhb22t3 xls3em1">...</div>
  (Session info: chrome=122.0.6261.70)
Stacktrace:
	GetHandleVerifier [0x00007FF63D94AD22+56930]
	(No symbol) [0x00007FF63D8BF622]
	(No symbol) [0x00007FF63D7742E5]
	(No symbol) [0x00007FF63D7C0A80]
	(No symbol) [0x00007FF63D7BE8AB]
	(No symbol) [0x00007FF63D7BC0B4]
	(No symbol) [0x00007FF63D7BAE45]
	(No symbol) [0x00007FF63D7AF798]
	(No symbol) [0x00007FF63D7DBC9A]
	(No symbol) [0x00007FF63D7AF09A]
	(No symbol) [0x00007FF63D7DBEB0]
	(No symbol) [0x00007FF63D7F81E2]
	(No symbol) [0x00007FF63D7DBA43]
	(No symbol) [0x00007FF63D7AD438]
	(No symbol) [0x00007FF63D7AE4D1]
	GetHandleVerifier [0x00007FF63DCC6AAD+3709933]
	GetHandleVerifier [0x00007FF63DD1FFED+4075821]
	GetHandleVerifier [0x00007FF63DD1817F+4043455]
	GetHan